In [2]:
# time series model for prediction of next dat temperature

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from sklearn.model_selection import train_test_split

In [4]:
# Load the dataset into the dataframe
# parse_date : is used to convert the date column into datetime objects
df = pd.read_csv("daily_minimum_temps.csv",parse_dates = ['Date'],index_col='Date')

/tmp/ipython-input-4-1747212286.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("daily_minimum_temps.csv",parse_dates = ['Date'],index_col='Date')


In [5]:
# checking the first
df.head()

,Temp
Date,
1981-01-01,20.7
1981-01-02,17.9
1981-01-03,18.8
1981-01-04,14.6
1981-01-05,15.8


In [6]:
df.isnull().sum()


,0
Temp,0


In [7]:
df.dtypes

,0
Temp,object


In [8]:
df.shape

(3650, 1)

In [9]:
from math import nan
# string format for a numric value can be
a='1'
stock = "one hundred three"
stock = nan
temp = "hello" # no sense
temp = nan

In [10]:
# converting the string values to numeric values by removing the double quotes and if string value does bot make any sense or it
# we convert that value to null
df['Temp'] = pd.to_numeric(df['Temp'],errors = 'coerce')

In [11]:
df.isnull().sum()

,0
Temp,3


In [12]:
df = df.dropna()

In [13]:
df.isnull().sum()

,0
Temp,0


In [14]:
from sklearn.preprocessing import MinMaxScaler

# Ensure 'Temp' is a column and not a method
print(df.columns)  # Should include 'Temp'

# Apply MinMax scaling
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df['Temp'].values.reshape(-1, 1))


Index(['Temp'], dtype='object')


In [15]:
#
seq_length = 30

In [16]:
# fucttion for creating sequence
def create_sequences(data_scaled,seq_length):
  X,y =[],[]
  for i in range(len(data_scaled)-seq_length):
    X.append(data_scaled[i:i+seq_length])
    y.append(data_scaled[i+seq_length])
  return np.array(X),np.array(y)



In [17]:
# create the function and storing the value in list X and y
X,y = create_sequences(data_scaled,seq_length)

In [18]:
# Divide the dataset into the train test and split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,shuffle = False)


In [19]:
#building the rnn model
model = Sequential([
    LSTM(64,activation = "relu",input_shape = (seq_length,1)),
    Dense(1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [20]:
from tensorflow.keras.losses import MeanSquaredError
model.compile(optimizer='adam', loss=MeanSquaredError())


In [21]:
# Train the model
model.fit(X_train,y_train,epochs =20,batch_size=32)

Epoch 1/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 0.0573
Epoch 2/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0106
Epoch 3/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0109
Epoch 4/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0107
Epoch 5/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0114
Epoch 6/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0103
Epoch 7/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0101
Epoch 8/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0100
Epoch 9/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0102
Epoch 10/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0094
Epoch 11/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.0097
Epoch 12/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.0096
Epoch 13/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0098
Epoch 14/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0096
Epoch 15/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0091
Epoc

In [22]:
y_pred_scaled = model.predict(X_test)

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step


In [23]:
#inverse transform the scaled data

y_pred_scaled = np.clip(y_pred_scaled,0,1)

y_pred = scaler.inverse_transform(y_pred_scaled)

y_test_actual = scaler.inverse_transform(y_test)

In [24]:
last_sequence = data_scaled[-seq_length:].reshape(1, seq_length, 1)
next_temp_scaled = model.predict(last_sequence)
next_temp_scaled = np.clip(next_temp_scaled, 0, 1)
next_day_temp = scaler.inverse_transform(next_temp_scaled)

print(f"Next day's temperature is : ", next_day_temp[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Next day's temperature is :  13.069314


In [25]:
model.save("model.h5")


In [26]:
import joblib
joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']